# 2D FEM Transient Simulatons of Hydrogen Absorption/Desorption from Reactor   

Simulates hydrogen absorption/desorption in a reactor. Reactor represented here as 2D rectangular domain. The mesh is generated using the Ferrite uniform mesg generator. 

##  Import Packages

In [1]:
using BlockArrays
using LinearAlgebra
using UnPack
using LinearSolve 
using SparseArrays
using Ferrite
using FerriteGmsh 
using OrdinaryDiffEq
using DifferentialEquations
using Plots 
using WriteVTK

## Section 1: Introduction 
We develop the following four transient 2D models: 

1. only time-evolution for solid concentration (unclear whether mass matrix should be integrated);
2. time-dependent diffusion of hydrogen gas and time-evolution for solid concentration;
3. time-dependent convection - diffusion of hydrogen gas and time-evolution for solid concentration;
  
More later. 

## Section 2: Mesh Generation
Common for various models in this notebook. 

In [46]:
nels  = (100, 25) # number of elements in each spatial direction
# nels  = (10, 4)
left  = Vec((0., 0.))    # start point for geometry 
right = Vec((1.0, 0.25,)) # end point for geometry
grid = generate_grid(Quadrilateral,nels,left,right);

## Section 3: Time-Evolution of Metal-Oxide Concentration Only  
Single field model. 

### Section 1.3: Assembly of Single Species Mass Matrix (possibly not required) 

In [47]:
function assemble_mass_matrix(cellvalues_u1::CellValues, M::SparseMatrixCSC, dh::DofHandler)
    n_basefuncs = getnbasefunctions(cellvalues_u1)
    Me = zeros(n_basefuncs, n_basefuncs)

    assembler = start_assemble(M)

    for cell in CellIterator(dh)

        fill!(Me, 0)

        Ferrite.reinit!(cellvalues_u1, cell)

        for q_point in 1:getnquadpoints(cellvalues_u1)
            dΩ = getdetJdV(cellvalues_u1, q_point)

            for i in 1:n_basefuncs
                v = shape_value(cellvalues_u1, q_point, i)
                for j in 1:n_basefuncs
                    u = shape_value(cellvalues_u1, q_point, j)
                    Me[i, j] += (v * u) * dΩ
                end
            end
        end

        assemble!(assembler, celldofs(cell), Me)
    end
    return M    
end

assemble_mass_matrix (generic function with 2 methods)

### Section 2.3: Spatial Discretization Set-Up and Mass Matrix Assembly

In [48]:
dim = 2 
degree = 1

# FE values
ip_u1 = Lagrange{RefQuadrilateral, degree}()
qr = QuadratureRule{RefQuadrilateral}(2*degree+1)
cellvalues_u1 = CellValues(qr, ip_u1);

dh = DofHandler(grid)

add!(dh, :u1, ip_u1)

close!(dh)

M = allocate_matrix(dh);
M = assemble_mass_matrix(cellvalues_u1, M, dh);

### Section 3.3: ODE System Set-Up and Time-Integration 
Should mass matrix be taken into account? 

In [49]:
# Scenario-1: absorption case: du > 0 : density of solid (amount of gas absorbed) increases until reaching saturation

# problem set-up: rhs function that defines the ODE
function rhs!(du,u,p,t)
    @. du .= (.5-u) # absorption assuming \rho_{sat} = 0.5 and multiplying factors equal 1
end 

# problem set-up: initial condition
uinit = zeros(ndofs(dh))
apply_analytical!(uinit, dh, :u1, x -> 0.3)

# problem set-up: set time span
Tend = 4
tspan = (0.0,Tend)

# associate mass matrix to the right-hand side function  
# rhs = ODEFunction(rhs!, mass_matrix=M)
rhs = ODEFunction(rhs!)

# problem set-up: define ODE problem 
problem = ODEProblem(rhs,uinit,tspan)

# solve: perform time integration 
timestepper = Rodas5P(autodiff=false)
sol = solve(problem,timestepper,reltol=1e-4,abstol=1e-4);

### Section 4.3: Time Integration and Post-Processing 

In [50]:
integrator = init(
    problem, timestepper; 
    adaptive=true, 
    progress=true, progress_steps=100,
    verbose=true,
);

pvd = paraview_collection("h2-storage")
for (step, (u,t)) in enumerate(intervals(integrator))
    display(t)
    VTKGridFile("h2-storage-$step", dh) do vtk
        write_solution(vtk, dh, u)
        pvd[t] = vtk
    end
end
vtk_save(pvd);

0.0

0.10851937903270505

0.42401868332435505

0.874682348488677

1.4390242062632592

2.165728712286362

3.0679809587055638

In [51]:
dim = 2 
degree = 1

# FE values
ip_u1 = Lagrange{RefQuadrilateral, degree}()
qr = QuadratureRule{RefQuadrilateral}(2*degree+1)
cellvalues_u1 = CellValues(qr, ip_u1);

ip_u2 = Lagrange{RefQuadrilateral, degree}()
qr = QuadratureRule{RefQuadrilateral}(2*degree+1)
cellvalues_u2 = CellValues(qr, ip_u2);

dh = DofHandler(grid)

add!(dh, :u1, ip_u1)
add!(dh, :u2, ip_u2)

close!(dh)

DofHandler{2, Grid{2, Quadrilateral, Float64}}
  Fields:
    :u1, Lagrange{RefQuadrilateral, 1}()
    :u2, Lagrange{RefQuadrilateral, 1}()
  Dofs per cell: 8
  Total dofs: 5252

In [52]:
length(grid.cells)

2500

In [53]:
dof_range_u1 = dof_range(dh, :u1); dof_range_u2 = dof_range(dh, :u2) 

5:8

In [33]:
range_u1 =  unique!(reduce(vcat, [ celldofs(dh, ce)[dof_range_u1] for ce in 1:length(grid.cells) ]))
range_u2 =  unique!(reduce(vcat, [ celldofs(dh, ce)[dof_range_u2] for ce in 1:length(grid.cells) ]))

55-element Vector{Int64}:
   5
   6
   7
   8
  11
  12
  15
  16
  19
  20
  23
  24
  27
   ⋮
  88
  91
  92
  94
  96
  98
 100
 102
 104
 106
 108
 110

## Section 4: Time Evolution of Metal Hydride Concentration ($u_1$) and H2 Gas ($u_2$)

### Section 1.4: Assembly of Two-Species Mass Matrix and Stiffness Matrix 

In [58]:
function assemble_mass_matrix(cellvalues_u1::CellValues, cellvalues_u2::CellValues, M::SparseMatrixCSC, dh::DofHandler)
    assembler = start_assemble(M)
    Me = zeros(ndofs_per_cell(dh), ndofs_per_cell(dh))
    range_u1 = dof_range(dh, :u1)
    ndofs_u1 = length(range_u1)
    range_u2 = dof_range(dh, :u2)
    ndofs_u2 = length(range_u2)
    u1 = Vector{Float64}(undef, ndofs_u1)
    u2 = Vector{Float64}(undef, ndofs_u2)
    
    for cell in CellIterator(dh)

        fill!(Me, 0)

        Ferrite.reinit!(cellvalues_u1, cell)

        for qp in 1:getnquadpoints(cellvalues_u1)
            dΩ = getdetJdV(cellvalues_u1, qp)
            for i in 1:ndofs_u1
                u1[i] = shape_value(cellvalues_u1, qp, i)
            end
            # u1-u1
            for (i, I) in pairs(range_u1), (j, J) in pairs(range_u1)
                Me[I, J] = (I==J)
            end
        end 
        
        Ferrite.reinit!(cellvalues_u2, cell)
        
        for qp in 1:getnquadpoints(cellvalues_u2)
            dΩ = getdetJdV(cellvalues_u2, qp)
            for i in 1:ndofs_u2
                u2[i] = shape_value(cellvalues_u2, qp, i)
            end
            # u2-u2 
            for (i, I) in pairs(range_u2), (j, J) in pairs(range_u2) 
                Me[I, J] += ( u2[i] * u2[j] ) * dΩ
            end
        end 
        
        assemble!(assembler, celldofs(cell), Me)
    end
    return M    
end

function assemble_stiffness_matrix(cellvalues_u1::CellValues, cellvalues_u2::CellValues, K::SparseMatrixCSC, dh::DofHandler)
    assembler = start_assemble(K)
    Ke = zeros(ndofs_per_cell(dh), ndofs_per_cell(dh))
    range_u1 = dof_range(dh, :u1)
    ndofs_u1 = length(range_u1)
    range_u2 = dof_range(dh, :u2)
    ndofs_u2 = length(range_u2)
    u1  = Vector{Float64}(undef, ndofs_u1)
    ∇u1 = Vector{Vec{2,Float64}}(undef, ndofs_u1)
    u2  = Vector{Float64}(undef, ndofs_u2)
    ∇u2 = Vector{Vec{2,Float64}}(undef, ndofs_u2)

    velocity = [1,0]
    
    for cell in CellIterator(dh)

        fill!(Ke, 0)

        Ferrite.reinit!(cellvalues_u1, cell)

        for qp in 1:getnquadpoints(cellvalues_u1)
            dΩ = getdetJdV(cellvalues_u1, qp)
            for i in 1:ndofs_u1
                u1[i] = shape_value(cellvalues_u1, qp, i)
                ∇u1[i] = shape_gradient(cellvalues_u1, qp, i)
            end
            # u2-u2
            for (i, I) in pairs(range_u2), (j, J) in pairs(range_u2) 
                Ke[I, J] += ( ∇u1[i] ⋅ ∇u1[j] - ( velocity ⋅ ∇u1[i] )* u1[j]) * dΩ
            end
        end 
            
        assemble!(assembler, celldofs(cell), Ke)
    end
    return K     
end

assemble_stiffness_matrix (generic function with 1 method)

### Section 2.4: Spatial Discretization Set-Up and Mass Matrix Assembly

In [59]:
dim = 2 
degree = 1

# FE values
ip_u1 = Lagrange{RefQuadrilateral, degree}()
qr = QuadratureRule{RefQuadrilateral}(2*degree+1)
cellvalues_u1 = CellValues(qr, ip_u1);

ip_u2 = Lagrange{RefQuadrilateral, degree}()
qr = QuadratureRule{RefQuadrilateral}(2*degree+1)
cellvalues_u2 = CellValues(qr, ip_u2);

dh = DofHandler(grid)

add!(dh, :u1, ip_u1)
add!(dh, :u2, ip_u2)

close!(dh)

dof_range_u1 = dof_range(dh, :u1)
dof_range_u2 = dof_range(dh, :u2)
range_u1 =  unique!(reduce(vcat, [ celldofs(dh, ce)[dof_range_u1] for ce in 1:length(grid.cells) ]))
range_u2 =  unique!(reduce(vcat, [ celldofs(dh, ce)[dof_range_u2] for ce in 1:length(grid.cells) ]))

# Boundary conditions 
ch = ConstraintHandler(dh)

# Boundary conditions 
wall = union(
    getfacetset(grid, "left"),
)
dbc = Dirichlet(:u2, wall, (x, t) -> [0.2])
add!(ch, dbc)
    
# Finalize
close!(ch)

M = allocate_matrix(dh);
M = assemble_mass_matrix(cellvalues_u1, cellvalues_u2, M, dh)
M[range_u1, range_u1] .= I(length(range_u1))

K = allocate_matrix(dh);
K = assemble_stiffness_matrix(cellvalues_u1, cellvalues_u2, K, dh);

### Section 3.4: ODE Set-up and Time-Integration  

In [60]:
function setup_initial_conditions!(uinit::Vector,dh)
   apply_analytical!(uinit, dh, :u1, x -> 0.3)
   apply_analytical!(uinit, dh, :u2, x -> 0.2)    
end;

struct RHSparams
    K::SparseMatrixCSC
    ch::ConstraintHandler   
    dh::DofHandler
    range_u1::Vector{Int64}
    range_u2::Vector{Int64}    
end

p = RHSparams(K,ch,dh,range_u1,range_u2)

# problem set-up: rhs function that defines the ODE
function rhs!(du,u,p::RHSparams,t)
    @unpack K,ch,dh,range_u1,range_u2 = p

    apply!(u, ch)

    # Linear contribution 
    mul!(du, -100*K, u) # du .= K * u    
    
    u1 = @view u[range_u1]
    u2 = @view u[range_u2] 
    @. du[range_u1] += (.5-u1)
    @. du[range_u2] += -(.5-u1)   
end 

# problem set-up: initial condition
uinit = zeros(ndofs(dh))
uinit = setup_initial_conditions!(uinit, dh)
apply!(uinit, ch)

# problem set-up: set time span
Tend = 10e0
tspan = (0.0,Tend)

# associate mass matrix to the right-hand side function  
# rhs = ODEFunction(rhs!, mass_matrix=M)
rhs = ODEFunction(rhs!)

# problem set-up: define ODE problem 
problem = ODEProblem(rhs,uinit,tspan, p)

# solve: perform time integration 
timestepper = Rodas5P(autodiff=false)
# sol = solve(problem,timestepper,reltol=1e-4,abstol=1e-4);

Rodas5P(; linsolve = nothing, precs = DEFAULT_PRECS,)

In [61]:
integrator = init(
    problem, timestepper; 
    adaptive=true, 
    progress=true, progress_steps=1,
    verbose=true,
);

pvd = paraview_collection("h2-storage")
for (step, (u,t)) in enumerate(intervals(integrator))
    display(t)
    VTKGridFile("h2-storage-$step", dh) do vtk
        write_solution(vtk, dh, u)
        pvd[t] = vtk
    end
end
vtk_save(pvd);

0.0

0.05936800419547967

0.10104744601682332

0.1743231673742664

0.29099980407657555

...

4.0485114156250654

4.99804739093424

6.1177070695080165

7.482350982163495

9.242735375834709